안녕하세요, DS 26기 박하늘입니다.

이번 과제의 목표는 **Unsloth 라이브러리를 이용해
Mistral-7B-Instruct 모델을 4-bit quantization + LoRA 방식으로 빠르게 파인튜닝**하는 것입니다.

####1. 환경
- 실행 환경 : **Google Colab 무료 T4 GPU**
- RAM / VRAM 요구 : 약 15 GB 이하
- 필수 패키지 : `unsloth bitsandbytes accelerate trl datasets`

####2. 절차
1. 데이터 선택
  - HF Datasets Hub 에서 도메인 특화 데이터셋 1개 선택

  - 예) jtatman/medical-sci-instruct-1m-sharegpt

2. 템플릿·매핑 설정

  - chat_template="mistral"

  - mapping={"role":"from","content":"value","user":"human","assistant":"gpt"}

3. 전처리 & LoRA 파인튜닝

  - 샘플 수 : 5 k ~ 10 k (자유)

  - LoRA r/α : 16/16
 learning_rate=2e-4 , 60~300 step (기본 세팅은 이러하나, 자유)

4. 전·후 모델 비교

  - 동일 질문 3~5개에 대해 Base vs Fine-tuned 출력 비교

5. 결과 요약문 작성 (Results 참고)



# 설치

In [1]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # 코랩 환경에서 실행하는 경우
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.7

# [TODO 0, 1] 모델 설정

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # 자유롭게 설정 가능
dtype = None # 자동 탐지의 경우 None. Tesla T4, V100 : Float16
load_in_4bit = True # 4 bit quantization 지원 여부, False로 설정 가능

# 4bit pre quantized 여러 모델을 지원하지만,,, mistral 7b를 사용 (다른 모델 사용하셔도 됩니다)
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
##TODO 0
##모델 내려받기

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # unsloth 모델이 아니어도 됩니다
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # unsloth quantized모델이 아닌 다른 gated model (예: Llama) 활용하는 경우 토큰 입력이 필요
)

# 파인튜닝 이후 모델과의 비교를 위해 학습 전 모델을 따로 복사
base_model, base_tok = FastLanguageModel.from_pretrained(
    model_name      = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length  = 2048,
    load_in_4bit    = True,
)
FastLanguageModel.for_inference(base_model)

==((====))==  Unsloth 2025.5.6: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

==((====))==  Unsloth 2025.5.6: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): M

In [4]:
##TODO 1
##LoRA hyperparameter 설정

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA rank : 8, 16, 32, 64, 128 중에 선정
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", # LoRA를 삽입할 모듈 설정(projection layer)
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # LoRA 가중치 scaling : supports any, 일반적으로 r 혹은 2r로 설정
    lora_dropout = 0, # LoRA dropout rate : supports any, but = 0 is optimized
    bias = "none",    # LoRA bias param 추가 여부 : any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # supports rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# [TODO 2, 3] 데이터 준비

**데이터셋 필드 확인**
- 데이터셋에 따라 chat_template, mapping을 적절히 바꿔줘야 합니다.

**chat_template="..."**
- 일반적으로 모델이 학습된 템플릿 형식과 동일한 템플릿 이름을 지정 (예: mistral 모델의 경우 "mistral")
- 새 포맷으로 파인튜닝하고 싶다면, 학습과 추론 과정에서 같은 포맷 써야 함

**mapping={...}**
- 데이터셋의 JSON/CSV 안의 필드 명이 다른 경우 매핑 수정 필요
- 예시) 데이터 셋이 {"speaker":"client","utterance":"..."} 이런 형태인 경우 다음과 같이 수정
- mapping={"role":"speaker","content":"utterance","user":"client","assistant":"bot"}

**데이터셋 크기가 너무 크다면...**
- 다음과 같이 일부만 사용해서 학습
- dataset = load_dataset(dataset_name, split = "train[:5000]")


**참고자료 (unsloth chat_template)**
- 참고 : https://github.com/unslothai/unsloth/wiki#chat-templates

In [5]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

In [6]:
dataset_name = "yahma/alpaca-cleaned"
raw_ds       = load_dataset(dataset_name, split="train[:6000]")

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral",
    mapping = {
        "role"      : "role",
        "content"   : "content",
        "user"      : "user",
        "assistant" : "assistant",
    },
    map_eos_token = True,
)

def formatting_prompts_func(examples):
    texts = []
    for instr, inp, out in zip(examples["instruction"],
                               examples["input"],
                               examples["output"]):
        user_msg = instr + ("\n" + inp if inp else "")
        convo = [
            {"role": "user",      "content": user_msg},
            {"role": "assistant", "content": out},
        ]
        txt = tokenizer.apply_chat_template(
            convo, tokenize=False, add_generation_prompt=False
        )
        texts.append(txt)
    return {"text": texts}

dataset = raw_ds.map(
    formatting_prompts_func,
    batched        = True,
    remove_columns = raw_ds.column_names,
)


README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [7]:
# 데이터 예시 출력 (매핑 후)
print(dataset[5]["text"])

<s>[INST] Write a concise summary of the following:
"Commodore 64 (commonly known as the C64 or CBM 64) was manufactured by Commodore Business Machine (CBM) in August 1982 with a starting price of $595. It was an 8-bit home computer with remarkable market success. Between 1983-1986, C64 sales amounted to about 17 million units sold, making them the best-selling single personal computer model of all time in 1983-1986. 

Additionally, the Commodore 64 dominated the market with between 30% and 40% share and 2 million units sold per year, outselling the IBM PC clones, Apple Computers, and Atari computers. Adding to their success, Sam Tramiel (former Atari president), during an interview in 1989, said they were building 400,000 C64s a month for a couple of years. " [/INST]The Commodore 64 was a highly successful 8-bit home computer manufactured by Commodore Business Machine (CBM) in 1982, with sales amounting to approximately 17 million units sold between 1983-1986. It dominated the market 

#### (If required) 커스텀 chat template

In [8]:
# 커스텀 chat template을 만드는 것도 가능함
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,),
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

# [TODO 4] Training

- huggingface Trl의 SFTTrainer 활용해서 training 진행
-[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 20,
        max_steps = 300,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/6000 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,000 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.617000
20,1.443100
30,1.154300
40,1.169800
50,1.116100
60,1.144900
70,1.057700
80,1.059500
90,1.052800
100,1.052800


# Inference


In [3]:
# ✅ 1. Unsloth 모델 import
from unsloth import FastLanguageModel

# ✅ 2. 두 모델 모두 inference 최적화
FastLanguageModel.for_inference(base_model)
FastLanguageModel.for_inference(model)

# ✅ 3. 비교 질문 (알파카 스타일 질문)
questions = [
    "How can I naturally lower my blood pressure?",
    "Is it okay to take ibuprofen on an empty stomach?",
    "Explain what a CT scan does in one sentence.",
    "Why do some people faint when they see blood?",
    "What are the benefits of drinking water regularly?",
]

# ✅ 4. 질문 → 답변 함수 정의
def ask_any(model, prompt, max_new_tokens=128):
    msgs = [{"role": "user", "content": prompt}]   # ← 여기 수정
    inputs = tokenizer.apply_chat_template(
        msgs,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        padding=False
    ).to("cuda")
    with torch.no_grad():
        out = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=max_new_tokens,
            use_cache=True,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True).strip()


# ✅ 5. 결과 비교 출력
for q in questions:
    print(f"\n🟡 𝗤: {q}\n")

    print("🔹 𝘽𝙖𝙨𝙚 𝙈𝙤𝙙𝙚𝙡:")
    print(ask_any(base_model, q))

    print("\n🔸 𝙁𝙞𝙣𝙚-𝙏𝙪𝙣𝙚𝙙 𝙈𝙤𝙙𝙚𝙡:")
    print(ask_any(model, q))

    print("-" * 100)


🟡 𝗤: How can I naturally lower my blood pressure?

🔹 𝘽𝙖𝙨𝙚 𝙈𝙤𝙙𝙚𝙡:
You can lower blood pressure by taking prescribed medication and consulting your doctor.

🔸 𝙁𝙞𝙣𝙚-𝙏𝙪𝙣𝙚𝙙 𝙈𝙤𝙙𝙚𝙡:
You can naturally lower blood pressure by reducing salt intake, exercising regularly, managing stress, and eating more fruits and vegetables.
----------------------------------------------------------------------------------------------------

🟡 𝗤: Is it okay to take ibuprofen on an empty stomach?

🔹 𝘽𝙖𝙨𝙚 𝙈𝙤𝙙𝙚𝙡:
Ibuprofen is a type of medicine used to treat pain.

🔸 𝙁𝙞𝙣𝙚-𝙏𝙪𝙣𝙚𝙙 𝙈𝙤𝙙𝙚𝙡:
It is generally not recommended to take ibuprofen on an empty stomach, as it can irritate the stomach lining. Taking it with food or milk helps reduce this risk.
----------------------------------------------------------------------------------------------------

🟡 𝗤: Explain what a CT scan does in one sentence.

🔹 𝘽𝙖𝙨𝙚 𝙈𝙤𝙙𝙚𝙡:
A CT scan is a medical tool.

🔸 𝙁𝙞𝙣𝙚-𝙏𝙪𝙣𝙚𝙙 𝙈𝙤𝙙𝙚𝙡:
A CT scan uses X-rays and computer processing to create det

# [TODO 5] Results

## 데이터 요약
- 데이터셋 이름: yahma/alpaca-cleaned
- 데이터셋 도메인(의료 포함한 범용 인스트럭션 지식)
- 데이터셋 중 학습에 활용한 샘플 수: 6,000개 (train[:6000])
- 템플릿, 매핑 설정 간단 요약:
  - 템플릿은 `mistral` chat-template 사용
  - `instruction` + `input`을 user 메시지로 합치고, `output`을 assistant 응답으로 매핑
  - {"role": "user", "content": ...}, {"role": "assistant", "content": ...} 형태로 구성 후 토크나이저 처리


## 파인튜닝
- 하이퍼 파라미터 변경사항:
  - **LoRA 설정**: r = 16, lora_alpha = 16, target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
  - **학습 배치**: per_device_train_batch_size = 2, gradient_accumulation_steps = 8 (effective batch size = 16)
  - **학습 스텝 및 워밍업**: max_steps = 300, warmup_steps = 20
  - **Optimizer 및 기타**: learning_rate = 2e-4, optim = "adamw_8bit", weight_decay = 0.01, lr_scheduler_type = "linear"
  - **기타 옵션**: packing = True, fp16 활성화 (T4 환경 기준)
- 학습 시간: 1:10:40 (Google Colab T4 환경 기준)
- training loss 마지막 값: 1.084


## 전후 예시 비교
- 예시 추론으로 본 Base 모델과 FT 모델 차이점:

Base model: 응답이 전반적으로 짧고 단편적이며, 정보가 누락되어 있거나 모호한 경우가 많았다. 예) "What are the benefits of drinking water regularly?" -> "Drinking water is important."

Fintuned model: 구체적이고, 설명이 길며 실제 정보를 포함했다. 예) 같은 질문에 대해 "Regular water intake helps regulate body temperature, maintain energy levels, support digestion, and flush out toxins from the body."



## (additional) 모델 저장과 로드

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoModelForPeftCausalLM
    from transformers import AutoTokenizer

    model = AutoModelForPeftCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")